# Predict Mask

<span style="font-size: 20px; line-height: 30px;">
<ol>
    <li> Load model </li>
    <li> Load test dataset - Images </li>
    <li> Predict the mask </li>
    <li> Calculate FPS </li>
</ol>
</span>

## Import

In [1]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope

## Seeding

In [2]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

## Hyperparameters

In [3]:
height = 256
width = 256

## Path

In [4]:
dataset_path = "dataset"
save_path = "prediction"

model_path = "files/unet.h5"

## Create folder

In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

## Folder to save predicted mask

In [6]:
create_dir(save_path)

## Load model

In [7]:
def dice_loss(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    dice = (2. * intersection + 1e-15) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1e-15)
    return 1.0 - dice

In [8]:
with CustomObjectScope({'dice_loss': dice_loss}):
    model = tf.keras.models.load_model(model_path)

In [9]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________________

## Load test dataset: Images

In [10]:
test_x = sorted(glob(os.path.join(dataset_path, "test", "images", "*")))
print("Test Images: ", len(test_x))

Test Images:  306


## Predict the Mask

In [11]:
time_taken = []

for x in tqdm(test_x):
    name = x.split("/")[-1]
    
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    h, w, _ = x.shape
    x = cv2.resize(x, (width, height))
    x = x/255.0
    x = np.expand_dims(x, axis=0)
    
    start_time = time.time()
    p = model.predict(x)[0]
    total_time = time.time() - start_time
    
    time_taken.append(total_time)
    
    p = cv2.resize(p, (w, h))
    p = p > 0.5
    p = p * 255.0
    
    cv2.imwrite(os.path.join(save_path, name), p)

100%|██████████| 306/306 [00:13<00:00, 23.26it/s]


In [12]:
mean_time = np.mean(time_taken)
mean_fps = 1/mean_time

print(f"Mean Time: {mean_time: 1.5f} - Mean FPS: {mean_fps:2.5f}")

Mean Time:  0.03625 - Mean FPS: 27.58312
